In [ ]:
def histo_hestetics(h):
    h.SetStats(0)
    h.GetYaxis().SetTitleSize(0.05)
    h.GetXaxis().SetTitleSize(0.05)


def canvas_hestetics_layout(c, n=3, n1=0):
    c.SetLeftMargin(0.02)
    c.SetBottomMargin(0.02)
    c.SetRightMargin(0.02)
    c.SetTopMargin(0.02)
    c.SetTicky()
    c.SetTickx()
    c.Divide(n, n1)
    for i in range(1, n+1):
        c.cd(i).SetBottomMargin(0.1)
        c.cd(i).SetLeftMargin(0.1)
        c.cd(i).SetRightMargin(0.13)


def canvas_hestetics_single(c):
    c.SetTicky()
    c.SetTickx()


In [ ]:
import fetch_output

hn = ["TaskRaw/hHits",
      "TaskRaw/hSlotPartMask",
      "TaskRaw/hIndexEOIsNoise",
      "TaskRaw/hIndexEOInTimeWin",
      "TaskRaw/hIndexEOHitRate",
      "TaskDigits/TOFRawsMulti",
      "TaskDigits/TOFRawHitMap",
      "TaskRaw/TOFRawsTime",
      "TaskRaw/OrbitDDL",
      "TaskRaw/mean_of_hits"]

for i in hn:
    fetch_output.main(ccdb_path=f"qc/TOF/MO/{i}",
                      out_path="Run505285",
                      preserve_ccdb_structure=False)


In [ ]:
from ROOT import TFile, TLatex, gPad, TH1F
from plotting import draw_nice_canvas
import os


def draw(run="505285"):
    names = ["hHits",
             "hSlotPartMask",
             "hIndexEOIsNoise",
             "hIndexEOInTimeWin",
             "hIndexEOHitRate",
             "TOFRawsMulti",
             "TOFRawHitMap",
             "TOFRawsTime",
             "OrbitDDL",
             "mean_of_hits"]
    h = {}

    for i in names:
        fn = f"Run{run}/{i}.root"
        if not os.path.isfile(fn):
            print("skipping", fn)
            continue
        f = TFile(fn, "READ")
        hn = "ccdb_object"
        histo = f.Get(hn)
        if not histo:
            print("Did not find", hn, "in", fn)
            if 0:
                f.ls()
            continue
        h[i] = histo
        h[i].SetDirectory(0)
    hmissing = {}

    def draw(name, opt=""):
        if not name in h:
            print(name, "is missing")
            hmissing[name] = TH1F(f"missing{name}",
                                  f"{name} is missing", 1, 0, 1)
            return hmissing[name]
        h[name].Draw(opt)
        return h[name]

    label = TLatex()
    label.SetTextFont(42)
    label.SetNDC()
    label.SetTextSize(0.06)
    label.SetTextAlign(22)
    label.SetTextAngle(0)

    cnoise = draw_nice_canvas("cnoise", 1600, 1250)
    canvas_hestetics_layout(cnoise, 0, 2)
    cnoise.cd(1).SetLogy()
    draw("hIndexEOHitRate", "HIST")
    label.DrawLatex(0.25, 0.8, f"Run {run}")
    cnoise.cd(2).SetLogy()
    draw("hIndexEOInTimeWin", "HIST").SetTitle("All channels, noisy in red")
    draw("hIndexEOIsNoise", "HIST SAME").SetLineColor(2)
    cnoise.SaveAs(f"Run{run}/noise_layout.png")

    craw = draw_nice_canvas("craw", 1800, 550)
    canvas_hestetics_layout(craw, 2)
    craw.cd(1).SetLogy()
    draw("hHits", "").SetLineWidth(2)
    label. DrawLatex(0.25, 0.8, f"Run {run}")
    craw.cd(2)
    draw("hSlotPartMask", "colz")
    craw.cd(3)
    craw.SaveAs(f"Run{run}/raw_layout.png")

    cdig = draw_nice_canvas("cdig", 1800, 850)
    canvas_hestetics_layout(cdig, 2, 2)
    cdig.cd(1).SetLogz()
    draw("TOFRawHitMap", "colz")
    label. DrawLatex(0.25, 0.8, f"Run {run}")
    cdig.cd(2)
    draw("TOFRawsMulti")
    cdig.cd(3)
    draw("TOFRawsTime")
    cdig.cd(4)
    draw("OrbitDDL", "colz")
    cdig.SaveAs(f"Run{run}/digits_layout.png")

    ctrend = draw_nice_canvas("ctrend", 1800, 800)
    draw("mean_of_hits")
    ctrend.SaveAs("trending.png")

    for i in h:
        csingle = draw_nice_canvas(f"single_{i}", logz=True)
        draw(draw, "colz HIST")
        label.DrawLatex(0.27, 0.8, f"Run {run}")
        csingle.SaveAs(f"Run{run}/{i}.png")
    return h, hmissing


R = draw()
print(R)
